# Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab
%matplotlib inline
from scipy.stats import norm

# Functions

In [ ]:
def rbe_wedenberg(dose, let, abx,q):
    """
    Wedenberg proton RBE model
    input parameters may be either numpy.array or scalars
    TODO: handle Cube() class directly
    :params dose: physical proton dose in [Gy]
    :params let: LETd in [keV/um] (protons only)
    :params abx: alpha_x / beta_x [Gy]
    :returns: RBE for the given parameters
    :ref: http://dx.doi.org/10.3109/0284186X.2012.705892
    """

    _apx = 1.000 + q * let / abx
    _sbpx = 1.0

    rbe = _rbe_apx(dose, _apx, _sbpx, abx)
    return rbe

In [ ]:
def _rbe_apx(dose, apx, sbpx, abx, dzero=0.0):
    """
    :params dose: proton dose      [Gy]
    :params apx: alpha_p / alpha_x [dimensionless] RBE_max = ap/ax when (dose -> 0 Gy)
    :params sbpx: beta_p / beta_x  [dimensionless] RBE_min = sqrt(bp/bx) when (dose -> inf Gy)
    :params abx: alpha_x / beta_x  [Gy]
    :params dzero: what to return in case of dose is zero (which would cause division by zero)
    """

    _rbe = 1.0 / (2.0 * dose)
    if hasattr(_rbe, '__iter__'):
        _rbe[_rbe == np.inf] = dzero
    else:
        if _rbe == np.inf:
            return dzero
    delta = abx * abx + 4. * abx * apx * dose + 4. * sbpx * sbpx * dose * dose
    delta *= (delta > 0)
    _rbe *= (np.sqrt(delta) - abx)
    return _rbe

In [ ]:
style_param = {'legend.fontsize': '16',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '18',
         'axes.titlesize':'26',
         'xtick.labelsize':'20',
         'ytick.labelsize':'20',
         'xtick.major.pad':'16',
         'ytick.major.pad':'16'}
    
pylab.rcParams.update(style_param)

# Open data

In [ ]:
open_fname = os.path.join('tmp','distrib_q.h5')
q = pd.read_hdf(open_fname, 'data_1')

In [ ]:
fname = os.path.join('data','sobp_shieldhit','output')
dose = os.path.join(fname,'dose.dat')
let=os.path.join(fname,'dlet.dat')

In [ ]:
dozym = os.path.join('data','doz_sobp.xlsx')
df_dozim = pd.read_excel(dozym,names=['depth','dose','depth2','dose2'])

In [ ]:
df_dose=pd.read_table(dose,names=['z','dose','err'],sep='\s+')
df_let=pd.read_table(let,names=['z','let','err'],sep='\s+')

In [ ]:
# LET and normalized dose in water phantom
dose_max=df_dose.dose[df_dose.dose>0.95*df_dose.dose.max()].mean()
rbe= pd.DataFrame(dtype='float')

#normalize dose to 2[Gy]
rbe["dose"]=df_dose['dose']*2/dose_max
rbe["let"]=df_let['let']
rbe["z"]=df_dose['z']

In [ ]:
mean_W, std_W = 0.434, 0.0301
mean_WA, std_WA = 0.389, 0.0218

In [ ]:
q_WA = np.random.normal(loc=mean_WA,scale=std_WA,size=50000)
q_W = np.random.normal(loc=mean_W,scale=std_W,size=50000)

In [ ]:
limit=df_dose.z[df_dose.dose<=df_dose.dose.max()*0.01].iloc[0]

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax.set_xlabel("Range [cm]",size=34, labelpad = 24)
ax.set_ylabel("Dose [Gy]",size=34, labelpad = 24)

ax.plot(rbe.z, rbe.dose,'green',markersize=1.2, label = "SHIELD-HIT12A Monte Carlo simulation")
ax.plot(df_dozim.depth2,df_dozim.dose2,'red',markersize=1.2, label = "Dosimetry")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major', labelsize=28)
ax.set_xlim(2,limit)
ax.set_ylim(0,3)

ax.legend(loc="center",bbox_to_anchor=(0.4, 0.9), borderaxespad=0.,fontsize=20)

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax2=ax.twinx()
ax.set_xlabel("Range [cm]",size=34, labelpad = 24)
ax.set_ylabel("Dose [Gy]",size=34, labelpad = 24)
ax2.set_ylabel("LET "+r"$\;[keV \cdot \mu m^{-1}]$",size=34,rotation=90)

ax.plot(rbe.z, rbe.dose,'green',markersize=1.2, label = "SHIELD-HIT12A \nMonte Carlo simulation \nDose")

ax2.plot(rbe.z, rbe.let,'k.',markersize=1.2, label = "SHIELD-HIT12A \nMonte Carlo simulation \n LET")
ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major', labelsize=28)


ax.set_xlim(2,limit)
ax2.set_xlim(2,limit)
ax.set_ylim(0,3)
ax.legend(loc="center",bbox_to_anchor=(0.25, 0.9), borderaxespad=0.,fontsize=20)
ax2.legend(loc="center",bbox_to_anchor=(0.25, 0.7), borderaxespad=0.,fontsize=20)

# Wedenberg RBE distribution for LET=2.

In [ ]:
tmp_WB = rbe_wedenberg(2.0,3., 2.0, q )
tmp_WA = pd.DataFrame(rbe_wedenberg(2.0,3., 2.0, q_WA))
tmp_W = pd.DataFrame(rbe_wedenberg(2.0,3., 2.0, q_W))

In [ ]:
WA_text = "Wedenberg extended model A \n"+str(np.round(tmp_WA.mean()[0],3))+"+- "+str(np.round(tmp_WA.std()[0]/tmp_WA.mean()[0]*100,2))+'%'
W_text = "Wedenberg model \n"+str(np.round(tmp_W.mean()[0],3))+"+- "+str(np.round(tmp_W.std()[0]/tmp_W.mean()[0]*100,2))+'%'
WB_text = "Wedenberg extended model B \n"+str(np.round(tmp_WB.mean()[0],3))+"+- "+str(np.round(tmp_WB.std()[0]/tmp_WB.mean()[0]*100,2))+'%'

In [ ]:
fig,ax = plt.subplots(figsize = (8,6))

tmp_W.plot.kde(ax=ax,color='r')
tmp_WA.plot.kde(ax=ax,color='blue')
tmp_WB.plot.kde(ax=ax,color='green')

ax.axvline(x=1.1, label='RBE = 1.1'.format(1.1),c='k',ls='--',lw=1.2)
ax.legend([W_text,WA_text,WB_text], bbox_to_anchor=(0.6, 0.9))

ax.set_title("")
ax.set_xlabel("\nRBE [-]")
ax.set_ylabel("Probability distribution [-]")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
#tmp=pd.DataFrame(list(zip(tmp_W,tmp_WA,tmp_WB)),
#                          columns=['rbe'])
#tmp.to_csv("results/figure_5_9")
#fig.savefig(fname="results/figure_5_9.png",dpi= 700)

In [ ]:
#RBE for q=0.434 from Wedenberg 85% CI
abx=2.
rbe['wed'] = rbe_wedenberg(rbe.dose, rbe.let, abx, mean_W)
rbe['wed_q_down'] = rbe_wedenberg(rbe.dose, rbe.let, abx, mean_W-1.44*std_W)
rbe['wed_q_up'] = rbe_wedenberg(rbe.dose, rbe.let, abx, mean_W+1.44*std_W)

# our RBE distribution

In [ ]:
dfr = (rbe.reset_index(inplace=False))

In [ ]:
#optional : n=q.size-1
n=q.size-1

In [ ]:
dfr =dfr.append([dfr]*n)

In [ ]:
dfr=(dfr.reset_index(inplace=False))[['z','let', 'dose']]

In [ ]:
dfr=dfr.sort_values(by=['z', 'let','dose'])

In [ ]:
dfr.set_index(['z'],inplace=True) 

In [ ]:
for name, group in dfr.groupby('z'):
    dfr.loc[(name), "q"] = q.values[:n+1]

In [ ]:
for name, group in dfr.groupby('z'):
    DOSE=group.dose.values
    LET=group.let.values
    Q=group.q.values  
    dfr.loc[(name), "rbe"] = rbe_wedenberg(DOSE, LET, abx, Q)

In [ ]:
dfr.reset_index(inplace=True)

In [ ]:
dfr_tmp= pd.DataFrame()
dfr_tmp["mediana"]=dfr.groupby('z').rbe.median()
dfr_tmp["meana"]=dfr.groupby('z').rbe.mean()
dfr_tmp["q_down"]=dfr.groupby('z').rbe.quantile(0.075)
dfr_tmp["q_up"]=dfr.groupby('z').rbe.quantile(0.925)

# Biological Dose in SOBP

In [ ]:
figure2_df= pd.DataFrame()

In [ ]:
figure2_df["Distance_cm"] = rbe.z
figure2_df["Ext_Model_median"] = dfr.groupby('z').rbe.median().values*rbe.dose
figure2_df["Ext_Model_mean"] = dfr.groupby('z').rbe.mean().values*rbe.dose

figure2_df["Wedenberg"] = rbe.wed*rbe.dose
figure2_df["Wedenberg_quantile_down"] = rbe.wed_q_down*rbe.dose
figure2_df["Wedenberg_quantile_up"] = rbe.wed_q_up*rbe.dose

figure2_df["Physical__Dose"] = rbe.dose
figure2_df["Dose_for_RBE_1_1"] = rbe.dose*1.1
figure2_df["Ext_Model_quantile_down"] = dfr.groupby('z').rbe.quantile(0.075).values*rbe.dose
figure2_df["Ext_Model_quantile_up"] = dfr.groupby('z').rbe.quantile(0.925).values*rbe.dose

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax2=ax.twinx()
ax.set_xlabel("Range [cm]",size=34, labelpad = 24)
ax.set_ylabel("Biological Dose [Gy(RBE)]",size=34, labelpad = 24)
ax2.set_ylabel("LET "+r"$\;[keV \cdot \mu m^{-1}]$",size=34,rotation=90)

ax.set_xlim(3,limit)
ax2.set_xlim(3,limit)

ax.set_ylim(0,3.5)
ax2.set_ylim(0,14)

ax.set_xticks(np.arange(3, limit, 1.0))

ax2.plot(rbe.z, rbe.let,'k.',markersize=1.2, label = "LET")

#ax.plot(rbe.z, rbe.dose,c='k',label="Dose [Gy]")
ax.plot(figure2_df.Distance_cm,figure2_df.Ext_Model_mean,'green',markersize=0.2, label= "Wedenberg extended model B")
ax.plot(figure2_df.Distance_cm,figure2_df.Wedenberg,'red', label = "Wedenberg model")
ax.plot(figure2_df.Distance_cm, figure2_df.Dose_for_RBE_1_1,'blue',markersize=0.1, label = "Dose for RBE 1.1") 

ax.scatter(figure2_df.Distance_cm,figure2_df.Ext_Model_quantile_down, s=0.1,c='green')
ax.scatter(figure2_df.Distance_cm,figure2_df.Ext_Model_quantile_up, s=0.1,c='green')

ax.scatter(figure2_df.Distance_cm,figure2_df.Wedenberg_quantile_down,s=0.1,c='red')
ax.scatter(figure2_df.Distance_cm,figure2_df.Wedenberg_quantile_up,s=0.1,c='red')


ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major', labelsize=28)

ax.legend(loc="center",bbox_to_anchor=(0.33, 0.9), borderaxespad=0.,fontsize=20)
ax2.legend(loc="center",bbox_to_anchor=(0.88, 0.95), borderaxespad=0.,fontsize=20)

In [ ]:
(figure2_df.Ext_Model_quantile_up - figure2_df.Ext_Model_quantile_down) / figure2_df.Ext_Model_mean

# IN SOBP

In [ ]:
def dose_in_sobp(range_insobp, title):
    fig,ax = plt.subplots(figsize = (10,6))

    df_W = pd.DataFrame(rbe_wedenberg(np.float64(rbe[rbe.z==range_insobp].dose.values), 
                        rbe[rbe.z == range_insobp].let.values, 
                        abx, q_W)*rbe[rbe.z == range_insobp].dose.values)
    
    df_W.plot.kde(ax=ax,color='red')
        
    df_WA = pd.DataFrame(rbe_wedenberg(np.float64(rbe[rbe.z==range_insobp].dose.values), 
                        rbe[rbe.z == range_insobp].let.values, 
                        abx, q_WA)*rbe[rbe.z == range_insobp].dose.values)
    
    df_WA.plot.kde(ax=ax,color='blue')

    df_WB = rbe_wedenberg(np.float64(rbe[rbe.z==range_insobp].dose.values), 
                        rbe[rbe.z == range_insobp].let.values, 
                        abx, q.q)*rbe[rbe.z == range_insobp].dose.values
    
    df_WB.plot.kde(ax=ax,color='green')
   

    ax.axvline(figure2_df[figure2_df.Distance_cm == range_insobp].Dose_for_RBE_1_1.values, c='black',ls='-.')

    
    W_text = "Wedenberg model\n"+str(np.round(df_W.mean()[0],3))+"+-"+str(np.round(df_W.std()[0]/df_W.mean()[0]*100,2))+"%"
    WA_text = "Wedenberg extended model A\n"+str(np.round(df_WA.mean()[0],3))+"+-"+str(np.round(df_WA.std()[0]/df_WA.mean()[0]*100,2))+"%"
    WB_text = "Wedenberg extended model B\n"+str(np.round(df_WB.mean(),3))+"+-"+str(np.round(df_WB.std()/df_WB.mean()*100,2))+"%"
    Clinic = "Dose for RBE = 1.1"
    
    ax.set_xlabel("\nBiological Dose [Gy(RBE)]")
    ax.set_ylabel("Probability Distribution [-]\n")
    ax.set_title(title)
    ax.grid()
    ax.minorticks_on()
    ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
    
    ax.legend([W_text,WA_text,WB_text, Clinic],bbox_to_anchor=(1.3, 1.05))

In [ ]:
dose_in_sobp(3.015, "Entrance Plateau")

In [ ]:
dose_in_sobp(7.015, "Middle SOBP")

In [ ]:
dose_in_sobp(8.515, "Distal SOBP")